In [19]:
import pickle 
import zipfile
import pandas as pd
import numpy as np 
import datetime
import pytz
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression


In [ ]:
# with zipfile.ZipFile('chgk.zip', 'r') as zip_ref:
#     zip_ref.extractall('./')

### Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
- взять в тренировочный набор турниры с dateStart из 2019 года; 
- в тестовый — турниры с dateStart из 2020 года.


In [2]:
with open('tournaments.pkl', 'rb') as fp:
    tour = pickle.load(fp)

with open('results.pkl', 'rb') as fp:
    res = pickle.load(fp)

with open('players.pkl', 'rb') as fp:
    pl = pickle.load(fp)
# players = pd.DataFrame.from_dict(a, orient='index')

In [3]:
df_pl = pd.DataFrame.from_dict(pl, orient='index')
df_tour = pd.DataFrame.from_dict(tour, orient='index')
df_res = pd.DataFrame.from_dict(res, orient='index')

In [4]:
year = []
for i in range(len(df_tour)):
    year.append(datetime.datetime.fromisoformat(df_tour['dateStart'].iloc[i]).astimezone(pytz.timezone('Europe/Moscow')).year)
df_tour['year'] = year

In [5]:
df_tour_train = df_tour[df_tour['year']==2019]
df_tour_test = df_tour[df_tour['year']==2020]

In [6]:
def res_mask(ids):
    data = []
    for id in ids:
        for t in res[id]:
            if ('mask' not in t) or (t['mask'] is None):
                continue

            position = t['position']
                    
            for pl in t['teamMembers']:
                pl = pl['player']
                data.append({
                    'tournament_id': id,
                    'team_id': t['team']['id'],
                    'team_name': t['team']['name'],
                    'player_id': pl['id'],
                    'team_position': position,
                    'player_name': f"{pl['surname']} {pl['name']} {pl['patronymic']}",
                    'mask': t['mask'],
                    'mask_len': len(t['mask'])
                })
    df = pd.DataFrame(data)

    return df

res_train = res_mask(df_tour_train.index)
res_test = res_mask(df_tour_test.index)

In [12]:
def filtrate(df):
    res_dict = {}
    for i, tournament in df.iterrows():
        existing_teams = []
        # for participant in res_train.iloc[i, :]:
        #     valid_participants.append(participant)
        # if valid_participants:
        #     valid_train_results[i] = valid_participants
        for team in res[i]:
            if team:
                if 'mask' in team and team['mask']:
                    if 'teamMembers' in team and team['teamMembers']:
                        existing_teams.append(team)
        if existing_teams:
            res_dict[i] = existing_teams
    return res_dict
    
res_train_dict = filtrate(df_tour_train)
res_test_dict = filtrate(df_tour_test)

In [14]:
def to_scv(df_name, target_name, res):
    trainx_f = open(f'{df_name}.csv', 'w')
    trainx_f.write('player_id,tournament_id,question_id\n')

    trainy_f = open(f'{target_name}.csv', 'w')
    trainy_f.write('answer\n')

    for tour_id in res:
        tournament = res[tour_id]
        for team in tournament:
            team_id = team['team']['id']
            for teammate in team['teamMembers']:
                pl_id = teammate['player']['id']
                for qu_id in range(1, len(team['mask']) + 1):
                    trainx_f.write(str(pl_id) + ',' + str(tour_id) + ',' + str(qu_id) + '\n')
                    trainy_f.write(team['mask'][qu_id - 1] + '\n')

    trainx_f.close()
    trainy_f.close()
    
to_scv('x_train', 'y_train', res_train_dict)
to_scv('x_test', 'y_test', res_test_dict)

### Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
- повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
- в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
- для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


In [15]:
df = pd.read_csv('x_train.csv')
target = pd.read_csv('y_train.csv')

/tmp/ipykernel_21895/2421535835.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  target = pd.read_csv('y_train.csv')


In [27]:
df_test = pd.read_csv('x_test.csv')
target_test = pd.read_csv('y_test.csv')

/tmp/ipykernel_21895/183924037.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  target_test = pd.read_csv('y_test.csv')


In [30]:
df_test.tournament_id.unique().shape

(173,)

In [17]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(df)
enc.categories_[0].shape

(59101,)

In [18]:
targets = []

for ans in target['answer']:
    if ans == '?' or ans == 'X': 
        targets.append(np.random.randint(0, 1))
    else:
        targets.append(int(ans))

targets = np.array(targets)

In [24]:
LogReg = LogisticRegression(solver='liblinear', C=10, n_jobs=-1)
LogReg.fit(enc.transform(df), targets)

/home/b_gainitdinov/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 40.
  warnings.warn(


LogisticRegression(C=10, n_jobs=-1, solver='liblinear')

In [32]:
y_test_predict_proba = [probas[0] for probas in LogReg.predict_proba(enc.transform(df_test))]

In [61]:
LogReg.coef_[0].shape
enc.transform(df)#.player_id.unique().shape

<21014267x60276 sparse matrix of type '<class 'numpy.float64'>'
	with 63042801 stored elements in Compressed Sparse Row format>

#### Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
- предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
- в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

In [62]:
# отранжируем участников
players = df.player_id.unique()
rating = pd.DataFrame({'id': players})
rating['skill'] = LogReg.coef_[0][:len(players)]
rating = rating.sort_values(by='skill', ascending=False)
rating = pd.merge(rating, df_pl, on="id").set_index('id')

In [33]:

def sum_score(mask):
    score = 0
    for ans in mask:
        if ans == '?' or ans == 'X':
            continue
        score += int(ans)
    return score

In [39]:
# посчитаем ранговые корреляции Спирмена и Кендалла
true_ratings = []
pred_ratings = []

k = 0
for tour_id in res_test_dict:
    tournament = res_test_dict[tour_id]

    true_rate = []
    pred_rate = []
    for team in tournament:
        mask_true = team['mask']
        correct_ans = sum_score(mask_true)

        predict_proba = [1 for _ in range(len(mask_true))]
        for player in team['teamMembers']:
            for qu_id in range(len(mask_true)):
                predict_proba[qu_id] *= y_test_predict_proba[k]
                k += 1

        true_rate.append((team['team']['id'], correct_ans / len(mask_true)))
        pred_rate.append((team['team']['id'], np.prod(predict_proba)))

    true_ratings.append(true_rate)
    pred_ratings.append(pred_rate)

In [79]:
from scipy.stats import spearmanr, kendalltau


spearmanr_coefs = []
kendalltau_coefs = []

for i in range(len(pred_ratings)):
    true_rating = true_ratings[i]
    true_rating.sort(key=lambda x:x[1])
    true_rating = [i[0] for i in true_rating]
    pred_rating = pred_ratings[i]
    pred_rating.sort(key=lambda x:x[1])
    pred_rating = [i[0] for i in pred_rating]
    coef, p = spearmanr(true_rating, pred_rating)
    spearmanr_coefs.append(abs(coef))
    coef, p = kendalltau(real_tournament_rating, predict_tournament_rating)
    kendalltau_coefs.append(abs(coef))
spearmanr_coefs = np.array(spearmanr_coefs)
spearmanr_coefs = spearmanr_coefs[~np.isnan(spearmanr_coefs)]
kendalltau_coefs = np.array(kendalltau_coefs)
kendalltau_coefs = kendalltau_coefs[~np.isnan(kendalltau_coefs)]

In [84]:
print(f'корреляция Спирмана : {np.array(spearmanr_coefs).mean()}\n\
корреляция Кендалла : {np.array(kendalltau_coefs).mean()}')

корреляция Спирмана : 0.18190958595599918
корреляция Кендалла : 0.047619047619047616
